In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

PNES_path ="./pilot/PNES/output/" # where the folders are saved
ES_path ="./pilot/ES/output/"


length = float('inf')
PNESdata = []
ESdata = []

In [2]:
for folder in os.listdir(PNES_path):
    index_path = os.path.join(PNES_path, folder) # where the jason files are saved
    if os.path.isdir(index_path):
        dataframe = pd.read_csv(os.path.join(index_path,'keypoints.csv'), header=None, skiprows=1)
        rolavg = dataframe.rolling(100).mean()
        rolavg = rolavg.dropna()
        rolavg = rolavg.reset_index(drop=True)
        PNESdata.append(rolavg.values)
        if rolavg.values.shape[0] < length:
            length = rolavg.values.shape[0]
            print(length)

3341
3301


In [3]:
for folder in os.listdir(ES_path):
    index_path = os.path.join(ES_path, folder) # where the jason files are saved
    if os.path.isdir(index_path):
        dataframe = pd.read_csv(os.path.join(index_path,'keypoints.csv'), header=None, skiprows=1)
        rolavg = dataframe.rolling(100).mean()
        rolavg = rolavg.dropna()
        rolavg = rolavg.reset_index(drop=True)
        ESdata.append(rolavg.values)
        if rolavg.values.shape[0] < length:
            length = rolavg.values.shape[0]
            print(length)

3181
2170


In [4]:
for idx, array in enumerate(PNESdata):
    PNESdata[idx] = array[:length,::]
    
PNESarray = np.vstack(PNESdata)
PNESarray = np.reshape(PNESarray, [PNESarray.shape[0], 1, PNESarray.shape[1]])
PNESlabel = np.reshape(np.zeros(PNESarray.shape[0]),[PNESarray.shape[0],1])

for idx, array in enumerate(ESdata):
    ESdata[idx] = array[:length,::]
    
ESarray = np.vstack(ESdata)
ESarray = np.reshape(ESarray,[ESarray.shape[0], 1, ESarray.shape[1]])
ESlabel = np.reshape(np.ones(ESarray.shape[0]),[ESarray.shape[0],1])

X = np.concatenate((PNESarray,ESarray),axis=0)
y = np.concatenate((PNESlabel,ESlabel),axis=0)
y = to_categorical(y)

In [5]:
print(PNESarray.shape)
print(ESarray.shape)
print(PNESlabel.shape)
print(ESlabel.shape)
print(X.shape)
print(y.shape)

(23870, 1, 75)
(34720, 1, 75)
(23870, 1)
(34720, 1)
(58590, 1, 75)
(58590, 2)


In [6]:
outfile = "./pilot/Datasets/OpenPose/frame_rolavg100.npz"
np.savez(outfile, X, y)